Lesson Plan for Plotly Lab
What is Python???

    Python is a programming language that is easy to learn and reads like plain english
    Important note is that in python, the most important thing are indents.  Improper indents WILL cause errors
Basic Types

    During this lessson plan we will be using a few different types
        String: a sequence of letters and symbols surrounded by ""
        Integer: whole number (eg 5)
        float: number with decimal places (eg 5.3)
        Object: a special type that has its own methods we can use - 
        a method is a series of lines of code that accomplishes some task
        Dataframe: special object imported by the pandas library, allows us
        to create a relational way of viewing and manipulating data
        Series/List: simply a list of some form of data
        Dictionary: a series of mappings from one type to another
        
What does Import do?

    the import statement gives us functions, objects or a set of objects and functions (known as a
    library) that other people created that we would like to use.
    the 'as' keyword lets us rename this library or object for our
    convienence
    
    CSV
        Gives us the ability to utilize and read in a comma seperated value
        file, such as the one I have provided
    Numpy
        Gives us the ability to create a series object, series objects are 
        what plotly uses to graph
     Plotly
        Gives us access to all of the methods we need to create a plotly graph
    Plotly.Plotly 
        This gives us easy access to plotly's plotting methods
    Plotly.graph_objs
        This is a plotly graph.  we are renaming it for ease later
    Pandas
        Gives us the ability to create a dataframe from our CSV files

Plotly tools set credentials

    in order to save our graphs to our specific account, Plotly wants to know our username and our specific API key
    We can find this in the following way
    go to https://plot.ly/ and either create an account or login
    hover over your account in the right hand corner and click settings 
    your username should be available in the left column
    click API keys on the left hand side
    click generate/regenerate key, and save that number
    set your credentials as I have done below.
    

In [ ]:
#setting up the project

import csv
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objs as go
import chart_studio
import pandas as pd
#username='Fake.Username',api_key='55555'
chart_studio.tools.set_credentials_file(username='charlestonf', api_key='Snch64bsUgs9gBHQWdNp')

In [27]:
#reading the data

csv_file = "univ_reduced.csv"
df = pd.read_csv(csv_file)
print(df) # do not include this statement if using a text editor

                                                 INSTNM  SAT_AVG_ALL     UGDS  \
0                              Alabama A & M University        850.0   4505.0   
1                   University of Alabama at Birmingham       1147.0  11269.0   
2                                    Amridge University          NaN    308.0   
3                   University of Alabama in Huntsville       1221.0   5829.0   
4                              Alabama State University        844.0   4740.0   
...                                                 ...          ...      ...   
7588  National Personal Training Institute of Cleveland          NaN      NaN   
7589  Bay Area Medical Academy - San Jose Satellite ...          NaN      NaN   
7590                        High Desert Medical College          NaN      NaN   
7591                        Vantage College-San Antonio          NaN      NaN   
7592  American Institute of Pharmaceutical Technolog...          NaN      NaN   

      UGDS_WHITE MD_EARN_WN

The variable CSV_file is going to be a string object that points to the location of our data

    the next line asks our pandas library to try to read our CSV file, and turn it into a dataframe.
    
When done successfully, it will be stored in the variable df.

    The print statement is used to see our data.
    Notice there are over 7500 rows
https://drive.google.com/drive/folders/1WCZQmB_dAHCr4lHE027qEcvMTPLchs8D?usp=sharing

Notice we have a lot of Nan values.  Plotly cannot graph those, so we need to get rid of them.
We do this by indexing into our dataframe using [], then selecting the column we want, and then keeping it if the value is not null
    
    we also have some "privacy surpressed" values in the salary category.  since this is not a number we want to 
    eliminate those as well
    the ~ operator means not.  
    we will tell our data frame to keep any values where the salary column does not contain the word "privacy surpressed"
    
    notice how now there are only a little over 1300 rows.  the rest of the datapoints contained 
    incomplete data that we could not graph.
    

In [28]:
#cleaning up the data

df = df[df.SAT_AVG_ALL.notnull()]
df = df[df.INSTNM.notnull()]
df = df[df.UGDS_WHITE.notnull()]
df = df[df.MD_EARN_WNE_P10.notnull()]
df = df[~df.MD_EARN_WNE_P10.str.contains('PrivacySuppressed')]
print(df) #do not include this statement if using a text editor

                                                 INSTNM  SAT_AVG_ALL     UGDS  \
0                              Alabama A & M University        850.0   4505.0   
1                   University of Alabama at Birmingham       1147.0  11269.0   
3                   University of Alabama in Huntsville       1221.0   5829.0   
4                              Alabama State University        844.0   4740.0   
5                             The University of Alabama       1181.0  31005.0   
...                                                 ...          ...      ...   
6820                                 Augusta University       1045.0   4773.0   
6821                    Middle Georgia State University        864.0   7225.0   
6866                        University of North Georgia       1013.0  16048.0   
6868  Northeastern University Professional Advanceme...       1435.0   2051.0   
6880                   Arizona State University-Skysong       1143.0  16219.0   

      UGDS_WHITE MD_EARN_WN

These lines extract the columns from our dataframe and give us arrays that we can use to graph
    We need to do this because plotly works with Numpy, not Pandas.
    to get the names out of our list, we look into our dataframe for the column we want, and call .values.  this simply gives us 
    back the values that were in that column.
    we want to know our university names, their average SAT, their starting salaries, and the percentage of white students.

In [29]:
#extracting data values

name = df['INSTNM'].values
sat_average = df['SAT_AVG_ALL'].values
salary = df['MD_EARN_WNE_P10'].values
percentage_white = df['UGDS_WHITE'].values

calling go.scatter3d is going to help us create a 3D scatter plot object containing all our points

    we want our x axis to be their SAT averages, so we set x = to the sat average
    we want our y values to be their starting salary, so we set y = salary
    we want our z values to be the school's percentage white, so we set z = percentage_white
    when we hover over a point, we want to be able to see what school it is, so we will set our text= to the name of the school
    Mode is how we want points displayed, for this example, we want our points displayed as markers (or dots)

Next we want to define what our markers look like, so we will send in a dictionary telling plotly how to format dots

    we are letting the size of the dots be 4 each
    we want to color our dots by the percentage of white students
    plotly has built in colorscales for coloring the points, for this example we will use Viridis
    Opacity is how transparent our dots we will be, we will set this to .8 (0 is invisible, 1 is solid)
    
To actually graph data, plotly can support many of these "traces" so it wants each trace sent into it as a list, 
    since we only have 1 trace for this example, we are creating a list with a single entry, trace 1

In [30]:
#preparing data for plotly

trace1 = go.Scatter3d(
    x=sat_average,
    y=salary,
    z=percentage_white,
    text=name,
    mode='markers',
    marker=dict(
        size=4,
        color=percentage_white, # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)
my_data = [trace1]

Next we want to define the layout of our graph

    we want the title of our graph to simply be university data, so we can set title to "university data"
    to label our axis, plotly asks us to create a scene where we define what we want our axis to be 
        
        inside of our scene, we can define the axis labels
        let us set our x axis label to SAT average
        let us set our y axis label to average salary
        let us set our z axis label to percentage white
        
Once we have all of that, we can actually plot the graph

    first we have to create a completed figure to graph.  Plotly will do this automatically, all it needs to know
    is the data we want to plot, and the layout, or how we want it to look.  
    so we specify that the data it should use is my_data, and the layout it should use is my_layout.
    
    Finally, in order to display the graph, we call py.plot or py.iplot.  and specify the graph, as well as what
    we want to save our graph as
    
    IMPORTANT if using a text editor use py.plot, iplot is only for jupyter notebook

In [31]:
my_layout = go.Layout(
    title='University Data',
    scene=dict(
        xaxis=dict(
            title='SAT Average'
        ),
        yaxis=dict(
            title='Average Salary'
        ),
        zaxis=dict(
            title='Percentage White'
        )
    )
)
fig = go.Figure(data=my_data, layout=my_layout)
py.iplot(fig, filename='univ_vis') #if using a text editor call py.plot(fig, filename='univ_vis')